# Applying Machine Learning and Deep Learning to identify home appliances consuming excess power

## Copyright (c) 2018, Faststream Technologies
## Author: Sudhanva Narayana

In [17]:
# Home Appliances Data
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [18]:
# Import Dataset
df_home_data_test = pd.read_csv('../../data/home_data_test.csv')
df_home_priority = pd.read_csv('../../data/home_priority.csv')
date_time_group = pd.read_csv('../../data/date_time_group.csv')

In [19]:
df_home_data_test.head()

,device,building,room,weather_type,date,from_time,to_time,time,no_of_people,time_stayed_mins,power
0,AC,1,101,cold,2018-01-03,13:00:00,14:00:00,afternoon,6,20,1919
1,Heater,1,101,cold,2018-01-18,10:00:00,11:00:00,morning,4,24,2593
2,Heater,1,101,cold,2018-01-22,18:00:00,19:00:00,evening,0,0,2054
3,Heater,1,101,cold,2018-01-22,19:00:00,20:00:00,night,3,1,2969
4,Refridgerator,1,101,cold,2018-01-17,08:00:00,09:00:00,morning,0,0,206


In [20]:
df_home_priority.head()

,device,weather_type,time,priority,order
0,AC,low cold,early morning,8,52
1,AC,low cold,morning,8,49
2,AC,low cold,afternoon,8,48
3,AC,low cold,evening,8,50
4,AC,low cold,night,8,51


In [21]:
date_time_group.head()

,date,from_time,power
0,01-01-2018,00:00:00,10841
1,01-01-2018,01:00:00,11672
2,01-01-2018,02:00:00,10856
3,01-01-2018,03:00:00,8587
4,01-01-2018,04:00:00,12275


In [22]:
total_power_consumption = df_home_data_test.groupby(['date', 'from_time']).sum()['power'].reset_index()